In [ ]:
#### %matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd 
from pandas import DataFrame, read_csv
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import numpy.matlib
from edward.models import (
    Categorical, Dirichlet, Empirical, InverseGamma,
    MultivariateNormalDiag, Normal, ParamMixture, Mixture, Logistic, Laplace, PointMass, Gamma)

X_data = pd.read_csv('./data/data2/X.txt', sep="\t", header = None)
Y1 = pd.read_csv('./data/data2/Y1.txt', sep="\t", header = None)
Y2 = pd.read_csv('./data/data2/Y2.txt', sep="\t", header = None)
Y3 = pd.read_csv('./data/data2/Y3.txt', sep="\t", header = None)
Y4 = pd.read_csv('./data/data2/Y4.txt', sep="\t", header = None)
Y5 = pd.read_csv('./data/data2/Y5.txt', sep="\t", header = None)
T_data = np.array(pd.read_csv('./data/data2/T.txt', sep="\t", header = None))

Y1 = np.array(Y1)/20
Y2 = np.array(Y2)/50
Y3 = np.array(Y3)
Y4 = np.array(Y4)
Y5 = np.array(Y5)/10
n0 = np.size(T_data,0)
m0 = np.size(T_data,1)

X_data = np.array(X_data)
Mask_T = (T_data>0)*1
T_data = np.array((T_data-60)/15)
T_data = np.array(Mask_T*(T_data))
tmp = np.reshape(np.array(Y3[:,0]),(n0,1))
X_data = np.concatenate((X_data,tmp),axis=1)
tmp = np.reshape(np.array(Y4[:,0]),(n0,1))
X_data = np.concatenate((X_data,tmp),axis=1)
D = np.size(X_data,1)
E1 = np.zeros([30,5])
E2 = np.zeros([30,5])
E5 = np.zeros([30,5])

for iter in range(0,20):
     # validation 
       
    index = np.arange(0,n0)
    print(index)
    #index = np.delete(index,idx)
    index_test = index[(iter)*145:(iter+1)*145]  
    index_val = index[(iter+1)*145:(iter+2)*145]  
    index_train = index
    index_train = np.delete(index_train, index_test)
  
    X_train = X_data[index_train,:]
    Y_train1 = Y1[index_train,0:8]
    Y_train2 = Y2[index_train,0:8]
    Y_train3 = Y3[index_train,0:8]
    Y_train4 = Y4[index_train,0:8]
    Y_train5 = Y5[index_train,0:8]
    T_train = T_data[index_train,0:8]    

    X_val = X_data[index_val,:]
    Y_val1 = Y1[index_val,0:8]
    Y_val2 = Y2[index_val,0:8]
    Y_val3 = Y3[index_val,0:8]
    Y_val4 = Y4[index_val,0:8]
    Y_val5 = Y5[index_val,0:8]
    T_val = T_data[index_val,0:8]    
   # index_test = idx
    T_test = T_data[index_test,0:8]
    Y_test1  = Y1[index_test,0:8]
    Y_test2  = Y2[index_test,0:8]
    Y_test5  = Y5[index_test,0:8]
    X_test = X_data[index_test,:]
 
    N = np.size(T_train,0)
    M = np.size(T_train,1)
    Ne = np.size(T_test,0)
    Mt = np.size(T_test,1)   
    D = np.size(X_test,1)
    
    err10 = 100
    err20 = 100
    err50 = 100
    Md = 5 # modality
    Mask_train = np.zeros((Md,N,M))
    Mask_train[0,:,:] = (Y_train1>0)*1
    Mask_train[1,:,:] = (Y_train2>0)*1
    Mask_train[2,:,:] = (Y_train3>0)*1
    Mask_train[3,:,:] = (Y_train4>0)*1
    Mask_train[4,:,:] = ((Y_train5-Y_train5)==0)*1
    Mask_test = np.zeros((Md-2,Ne,Mt))
    Mask_test[0,:,:] = (Y_test1>0)*1
    Mask_test[1,:,:] = (Y_test2>0)*1
    Mask_test[2,:,:] = ((Y_test5-Y_test5)==0)*1      
    Mask_val = np.zeros((Md-2,Ne,Mt))
    Mask_val[0,:,:] = (Y_val1>0)*1
    Mask_val[1,:,:] = (Y_val2>0)*1
    Mask_val[2,:,:] = ((Y_val5-Y_val5)==0)*1  
    for i_val in range(0,10): 
        X = tf.placeholder(tf.float32, [N,D])
        T = tf.placeholder(tf.float32, [N,M])
        Mask = tf.placeholder(tf.float32, [Md,N,M])
        Yhat1 = tf.placeholder(tf.float32, [N,M])
        Yhat2 = tf.placeholder(tf.float32, [N,M])
        Yhat3 = tf.placeholder(tf.float32, [N,M])
        Yhat4 = tf.placeholder(tf.float32, [N,M])
        Yhat5 = tf.placeholder(tf.float32, [N,M])
        sigma_q = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([Md,1]))))
        sigma_s = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([Md,1]))))
        sigma_y = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([Md,1]))))
        sigma_c = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([1]))))
        sigma_h = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([1]))))
      # parameters ininitialization
        h = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([Md]))))
        c = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([Md]))))
        w = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([D,1]))))
        b = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([1]))))
        w1 = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([D,1]))))
        b1 = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([1]))))
        v = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([D,1]))))
        a = tf.sqrt(tf.exp(tf.Variable(tf.random_normal([D,1]))))
          
   # models
        sm1 = Normal(loc=(tf.matmul(X,w) + b),scale = sigma_s[0]*tf.ones([N,1],tf.float32))
        sm2 = Normal(loc=(tf.matmul(X,w) + b),scale = sigma_s[1]*tf.ones([N,1],tf.float32)) 
        sm3 = Normal(loc=(tf.matmul(X,w) + b),scale = sigma_s[2]*tf.ones([N,1],tf.float32)) 
        sm4 = Normal(loc=(tf.matmul(X,w) + b),scale = sigma_s[3]*tf.ones([N,1],tf.float32)) 
        sm5 = Normal(loc=(tf.matmul(X,w) + b),scale = sigma_s[4]*tf.ones([N,1],tf.float32))
        
        qm1 = Normal(loc=tf.matmul(X,v) + a[0],scale = sigma_q[0]*tf.ones([N,1],tf.float32)) 
        qm2 = Normal(loc=tf.matmul(X,v) + a[1],scale = sigma_q[1]*tf.ones([N,1],tf.float32)) 
        qm3 = Normal(loc=tf.matmul(X,v) + a[2],scale = sigma_q[2]*tf.ones([N,1],tf.float32)) 
        qm4 = Normal(loc=tf.matmul(X,v) + a[3],scale = sigma_q[3]*tf.ones([N,1],tf.float32)) 
        qm5 = Normal(loc=tf.matmul(X,v) + a[4],scale = sigma_q[4]*tf.ones([N,1],tf.float32)) 
        
        Z1 = tf.sigmoid(sm1*(T-qm1))*Mask[0,:,:]     
        Z2 = tf.sigmoid(sm2*(T-qm2))*Mask[1,:,:]      
        Z3 = tf.sigmoid(sm3*(T-qm3))*Mask[2,:,:]     
        Z4 = tf.sigmoid(sm4*(T-qm4))*Mask[3,:,:]      
        Z5 = tf.sigmoid(sm5*(T-qm5))*Mask[4,:,:]      
      
        Yhat1 = Normal(loc=(-c[0]*Z1 + h[0])*Mask[0,:,:],scale = sigma_y[0]*tf.ones([N,M],tf.float32))
        Yhat2 = Normal(loc=(c[1]*Z2 + h[1])*Mask[1,:,:],scale = sigma_y[1]*tf.ones([N,M],tf.float32))
        Yhat3 = Normal(loc=(c[2]*Z3 + h[2])*Mask[2,:,:],scale = sigma_y[2]*tf.ones([N,M],tf.float32))
        Yhat4 = Normal(loc=(c[3]*Z4 + h[3])*Mask[3,:,:],scale = sigma_y[3]*tf.ones([N,M],tf.float32))
        Yhat5 = Normal(loc=(c[4]*Z5 + h[4])*Mask[4,:,:],scale = sigma_y[4]*tf.ones([N,M],tf.float32))
        
        qc1 =  Normal(loc=tf.Variable(tf.random_normal([1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))
        qh1 =  Normal(loc=tf.Variable(tf.random_normal([1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))        
        qq1 = Normal(loc=tf.Variable(tf.random_normal([N,1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([N,1]))))
        qq2 = Normal(loc=tf.Variable(tf.random_normal([N,1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([N,1]))))
        qq3 = Normal(loc=tf.Variable(tf.random_normal([N,1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([N,1]))))
        qq4 = Normal(loc=tf.Variable(tf.random_normal([N,1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([N,1]))))
        qq5 = Normal(loc=tf.Variable(tf.random_normal([N,1])),
             scale=tf.nn.softplus(tf.Variable(tf.random_normal([N,1]))))
        latent_vars = {qm1:qq1, qm2:qq2,qm3:qq3,qm4:qq4,qm5:qq5}
        data = {X:X_train, T:T_train, Yhat1:Y_train1, Yhat2:Y_train2, Yhat3:Y_train3, 
                Yhat4:Y_train4,Yhat5:Y_train5, Mask:Mask_train}        
        inference = ed.KLqp(latent_vars,data)
        inference.run(n_iter=50000)  
        X_val =  tf.cast(X_val,tf.float32)
        q11 = tf.matmul(X_val,v) + a[0]
        q22 = tf.matmul(X_val,v) + a[1]
        q55 = tf.matmul(X_val,v) + a[4]
        s1 = tf.matmul(X_val,w) + b
        s2 = tf.matmul(X_val,w) + b
        s5 = tf.matmul(X_val,w) + b
        zval1 = Mask_val[0,:,:]*(tf.sigmoid((T_val-q11)*s1))
        Yval1 = -c[0]*zval1+h[0]
        zval2 = Mask_val[1,:,:]*tf.sigmoid((T_val-q22)*s2) 
        Yval2 = c[1]*zval2+h[1]
        zval5 = Mask_val[2,:,:]*tf.sigmoid((T_val-q55)*s5) 
        Yval5 = c[4]*zval5+h[4]
        n1 = tf.count_nonzero(Mask_val[0,:,:])
        n2 = tf.count_nonzero(Mask_val[1,:,:])    
        n5 = tf.count_nonzero(Mask_val[2,:,:])  
        err1 = (abs(Mask_val[0,:,:]*(Y_val1[:,:]-Yval1[:,:])))
        err2 = (abs(Mask_val[1,:,:]*(Y_val2[:,:]-Yval2[:,:])))
        err5 = (abs(Mask_val[2,:,:]*(Y_val5[:,:]-Yval5[:,:])))
        err1 = tf.reduce_sum(err1)
        err2 = tf.reduce_sum(err2)
        err5 = tf.reduce_sum(err5)
        err1 = err1.eval()/n1.eval()
        err2 = err2.eval()/n2.eval()
        err5 = err5.eval()/n5.eval()
        if (err1+err2+err5)<(err10+err20+err50):
              v0 = v
              w0 = w
              a0 = a
              b0 = b
              c0 = c
              h0 = h
              sigma_s0 = sigma_s
              sigma_q0 = sigma_q
              sigma_y0 = sigma_y
              err10 = err1
              err20 = err2
              err50 = err5
              print(err1)
              print(err2)
              print(err5)
# testing using MAP to estimate s and q
# construct the testing graph model
    for iter_tt in range(1,5):
        Me = iter_tt       
        print("known time points")
        print(Me)
        Xt = tf.placeholder(tf.float32, [Ne,D])
        Tt = tf.placeholder(tf.float32, [Ne,Me])  
        Maskt = tf.placeholder(tf.float32, [Md-2,Ne,Me])
        Yt1 = tf.placeholder(tf.float32,[Ne,Me])
        Yt2 = tf.placeholder(tf.float32,[Ne,Me])    
        Yt5 = tf.placeholder(tf.float32,[Ne,Me])  
# models   

        s1 =  Normal(loc=(tf.matmul(Xt,w0) + b0), scale = sigma_s0[0]*tf.ones([Ne,1],tf.float32))
        s2 =  Normal(loc=(tf.matmul(Xt,w0) + b0), scale = sigma_s0[1]*tf.ones([Ne,1],tf.float32))
        s5 =  Normal(loc=(tf.matmul(Xt,w0) + b0), scale = sigma_s0[4]*tf.ones([Ne,1],tf.float32))
        q11 = Normal(loc=tf.matmul(Xt,v0) + a0[0], scale=sigma_q0[0]*tf.ones([Ne,1],tf.float32))
        q12 = Normal(loc=tf.matmul(Xt,v0) + a0[1], scale=sigma_q0[1]*tf.ones([Ne,1],tf.float32))
        q15 = Normal(loc=tf.matmul(Xt,v0) + a0[4], scale=sigma_q0[4]*tf.ones([Ne,1],tf.float32))
        
        q_s1 =  Normal(loc=tf.Variable(tf.random_normal([Ne,1])),
               scale=tf.nn.softplus(tf.Variable(tf.random_normal([Ne,1]))))    
        q_s2 = Normal(loc=tf.Variable(tf.random_normal([Ne,1])),
               scale=tf.nn.softplus(tf.Variable(tf.random_normal([Ne,1])))) 
        q_s5 = Normal(loc=tf.Variable(tf.random_normal([Ne,1])),
               scale=tf.nn.softplus(tf.Variable(tf.random_normal([Ne,1])))) 
        q_q11 = Normal(loc=tf.Variable(tf.random_normal([Ne,1])),
               scale=tf.nn.softplus(tf.Variable(tf.random_normal([Ne,1]))))
        q_q22 = Normal(loc=tf.Variable(tf.random_normal([Ne,1])),
               scale=tf.nn.softplus(tf.Variable(tf.random_normal([Ne,1]))))
        q_q55 = Normal(loc=tf.Variable(tf.random_normal([Ne,1])),
               scale=tf.nn.softplus(tf.Variable(tf.random_normal([Ne,1]))))
        
        Zp1 = tf.sigmoid(s1*(Tt-q11))*Maskt[0,:,:]    
        Zp2 = tf.sigmoid(s2*(Tt-q12))*Maskt[1,:,:]       
        Zp5 = tf.sigmoid(s5*(Tt-q15))*Maskt[2,:,:]       
        Yt1 = Normal(loc=(-c0[0]*Zp1+h0[0])*Maskt[0,:,:],scale = sigma_y0[0]*tf.ones([Ne,Me],tf.float32))
        Yt2 = Normal(loc=(c0[1]*Zp2+h0[1])*Maskt[1,:,:],scale = sigma_y0[1]*tf.ones([Ne,Me],tf.float32))
        Yt5 = Normal(loc=(c0[4]*Zp5+h0[4])*Maskt[2,:,:],scale = sigma_y0[4]*tf.ones([Ne,Me],tf.float32))
        data = {Xt:X_test, Tt:T_test[:,0:Me], Maskt:Mask_test[:,:,0:Me], Yt1:Y_test1[:,0:Me], Yt2:Y_test2[:,0:Me],
               Yt5: Y_test5[:,0:Me]}
        inference = ed.KLqp({s1:q_s1, s2:q_s2, s5:q_s5, q11:q_q11, q12:q_q22, q15:q_q55},data) 
        inference.run(n_iter=50000)  
        
        zhat1 = Mask_test[0,:,:]*(tf.sigmoid((T_test-q_q11)*q_s1))
        Yhat1 = -c0[0]*zhat1+h0[0]
        zhat2 = Mask_test[1,:,:]*tf.sigmoid((T_test-q_q22)*q_s2) 
        Yhat2 = c0[1]*zhat2+h0[1]
        zhat5 = Mask_test[2,:,:]*tf.sigmoid((T_test-q_q55)*q_s5) 
        Yhat5 = c0[4]*zhat5+h0[4]
        err1 = (abs(Mask_test[0,:,Me:Mt]*(Y_test1[:,Me:Mt]-Yhat1[:,Me:Mt])))
        err2 = (abs(Mask_test[1,:,Me:Mt]*(Y_test2[:,Me:Mt]-Yhat2[:,Me:Mt])))
        err11 = tf.reduce_sum(err1)*20
        err22 = tf.reduce_sum(err2)*50
        err5 = (abs(Mask_test[2,:,Me:Mt]*(Y_test5[ :,Me:Mt]-Yhat5[:,Me:Mt])))
        err55 = tf.reduce_sum(err5)*10    
        n1 = tf.count_nonzero(Mask_test[0,:,Me:Mt])
        n2 = tf.count_nonzero(Mask_test[1,:,Me:Mt])    
        n5 = tf.count_nonzero(Mask_test[2,:,Me:Mt])   
        E1[iter,iter_tt] = err11.eval()/n1.eval() 
        E2[iter,iter_tt] = err22.eval()/n2.eval()
        E5[iter,iter_tt] = err55.eval()/n5.eval()        
        np.savetxt( str(iter) + str(iter_tt)+"E1.txt", E1, delimiter=",")
        np.savetxt( str(iter) + str(iter_tt)+"E2.txt", E2, delimiter=",")
        np.savetxt( str(iter) + str(iter_tt)+"E5.txt", E5, delimiter=",")
   
    X_test =  tf.cast(X_test,tf.float32)
    q11 = tf.matmul(X_test,v0) + a[0]
    q22 = tf.matmul(X_test,v0) + a0[1]
    q55 = tf.matmul(X_test,v0) + a0[4]
    s1 = tf.matmul(X_test,w0) + b0
    s2 = tf.matmul(X_test,w0) + b0
    s5 = tf.matmul(X_val,w0) + b0
    zt1 = Mask_test[0,:,:]*(tf.sigmoid((T_test-q11)*s1))
    Yt1 = -c0[0]*zt1+h0[0]
    zt2 = Mask_test[1,:,:]*tf.sigmoid((T_test-q22)*s2) 
    Yt2 = c0[1]*zt2+h0[1]
    zt5 = Mask_test[2,:,:]*tf.sigmoid((T_test-q55)*s5) 
    Yt5 = c0[4]*zt5+h0[4]
    n1 = tf.count_nonzero(Mask_test[0,:,:])
    n2 = tf.count_nonzero(Mask_test[1,:,:])    
    n5 = tf.count_nonzero(Mask_test[2,:,:])  
    err1 = (abs(Mask_test[0,:,:]*(Yt1[:,:]-Y_test1[:,:])))
    err2 = (abs(Mask_test[1,:,:]*(Yt2[:,:]-Y_test2[:,:])))
    err5 = (abs(Mask_test[2,:,:]*(Yt5[:,:]-Y_test5[:,:])))
    err1 = tf.reduce_sum(err1)*20
    err2 = tf.reduce_sum(err2)*50
    err5 = tf.reduce_sum(err5)*10
    err11 = err1.eval()/n1.eval()
    err22 = err2.eval()/n2.eval()
    err55 = err5.eval()/n5.eval()
    E1[iter,0] = err11
    E2[iter,0] = err22
    E5[iter,0] = err55
    print("Mean absolute error on test data:")        
    print(E1)
    print(E2) 
    print(E5)
    np.savetxt( str(iter) + str(0)+"E1.txt", E1, delimiter=",")
    np.savetxt( str(iter) + str(0)+"E2.txt", E2, delimiter=",")
    np.savetxt( str(iter) + str(0)+"E5.txt", E5, delimiter=",")

/home/zyy/.conda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater


[   0    1    2 ..., 3054 3055 3056]
50000/50000 [100%] ██████████████████████████████ Elapsed: 505s | Loss: -393132.281
0.146479781924
0.218751628305
0.100650668966
50000/50000 [100%] ██████████████████████████████ Elapsed: 509s | Loss: -380363.781
47000/50000 [ 94%] ████████████████████████████   ETA: 30s | Loss: -393159.938